In [82]:
#TODO - dodać opisy, komentarze, uporzdkować notebook przed oddaniem

In [104]:
import kaggle
import pandas as pd
import numpy as np

from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error


#### Fetch S&P 500 data from Kaggle ☁️

In [84]:
a = kaggle.api.dataset_download_cli('hiteshyadavx/s-and-p-500-stock-prices')

from zipfile import ZipFile
with ZipFile('s-and-p-500-stock-prices.zip', 'r') as zipObj:
   zipObj.extractall()

Dataset URL: https://www.kaggle.com/datasets/hiteshyadavx/s-and-p-500-stock-prices
License(s): CC0-1.0
s-and-p-500-stock-prices.zip: Skipping, found more recently modified local copy (use --force to force download)


In [85]:
df = pd.read_csv('Prices.csv')
df.head()

,Date,AAPL,AMZN,GOOGL,JNJ,JPM,META,MSFT,NVDA,PG,XOM
0,2015-01-02,24.237549,15.4260,26.278944,76.955551,46.511124,77.905800,39.858452,0.483011,66.518349,57.916889
1,2015-01-05,23.554735,15.1095,25.778227,76.418083,45.067196,76.654541,39.491920,0.474853,66.202065,56.332203
2,2015-01-06,23.556952,14.7645,25.142035,76.042580,43.898651,75.621750,38.912289,0.460457,65.900513,56.032726
3,2015-01-07,23.887281,14.9210,25.068092,77.721291,43.965630,75.621750,39.406689,0.459257,66.246216,56.600468
4,2015-01-08,24.805077,15.0230,25.155436,78.332405,44.948116,77.637688,40.565952,0.476533,67.003761,57.542580


In [86]:
df.info()
df = df.set_index("Date")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2765 entries, 0 to 2764
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    2765 non-null   object 
 1   AAPL    2765 non-null   float64
 2   AMZN    2765 non-null   float64
 3   GOOGL   2765 non-null   float64
 4   JNJ     2765 non-null   float64
 5   JPM     2765 non-null   float64
 6   META    2765 non-null   float64
 7   MSFT    2765 non-null   float64
 8   NVDA    2765 non-null   float64
 9   PG      2765 non-null   float64
 10  XOM     2765 non-null   float64
dtypes: float64(10), object(1)
memory usage: 237.7+ KB


In [87]:
summary_stats = pd.DataFrame({
    "mean_price": df.mean(),
    "median_price": df.median(),
    "std_price": df.std()
})

print(summary_stats)

       mean_price  median_price   std_price
AAPL   106.264792     88.466438   74.267900
AMZN   111.714225     99.740997   61.701738
GOOGL   93.641694     73.245079   57.406736
JNJ    124.893336    125.980705   29.992217
JPM    119.287875     99.122780   65.749924
META   261.113889    188.840881  173.768333
MSFT   206.076353    191.971481  143.224672
NVDA    32.758861      9.494388   49.786936
PG     107.687909    108.853691   36.761664
XOM     68.824275     57.200081   25.336264


In [88]:
df.index = pd.to_datetime(df.index)
df.head()

,AAPL,AMZN,GOOGL,JNJ,JPM,META,MSFT,NVDA,PG,XOM
Date,,,,,,,,,,
2015-01-02,24.237549,15.4260,26.278944,76.955551,46.511124,77.905800,39.858452,0.483011,66.518349,57.916889
2015-01-05,23.554735,15.1095,25.778227,76.418083,45.067196,76.654541,39.491920,0.474853,66.202065,56.332203
2015-01-06,23.556952,14.7645,25.142035,76.042580,43.898651,75.621750,38.912289,0.460457,65.900513,56.032726
2015-01-07,23.887281,14.9210,25.068092,77.721291,43.965630,75.621750,39.406689,0.459257,66.246216,56.600468
2015-01-08,24.805077,15.0230,25.155436,78.332405,44.948116,77.637688,40.565952,0.476533,67.003761,57.542580


In [90]:
np.isinf(df.select_dtypes(np.number)).sum().sum()

(df == 0).sum().sort_values(ascending=False).head(10)

AAPL     0
AMZN     0
GOOGL    0
JNJ      0
JPM      0
META     0
MSFT     0
NVDA     0
PG       0
XOM      0
dtype: int64

In [91]:
df = df.astype("float32")
df.dtypes

AAPL     float32
AMZN     float32
GOOGL    float32
JNJ      float32
JPM      float32
META     float32
MSFT     float32
NVDA     float32
PG       float32
XOM      float32
dtype: object

In [92]:
df.describe(percentiles=[0.01, 0.99])

,AAPL,AMZN,GOOGL,JNJ,JPM,META,MSFT,NVDA,PG,XOM
count,2765.000000,2765.000000,2765.000000,2765.000000,2765.000000,2765.000000,2765.000000,2765.000000,2765.000000,2765.000000
mean,106.264793,111.714218,93.641701,124.893333,119.287888,261.113892,206.076355,32.758865,107.687904,68.824280
std,74.267921,61.701744,57.406746,29.992214,65.749908,173.768295,143.224640,49.786922,36.761639,25.336252
min,20.604078,14.347500,24.666626,68.278831,40.474892,73.536331,34.437153,0.459257,51.235596,24.145136
1%,21.556286,18.528000,26.690235,71.350150,43.369136,77.018397,36.098407,0.480816,54.479854,27.340117
50%,88.466438,99.740997,73.245079,125.980705,99.122780,188.840881,191.971481,9.494388,108.853691,57.200081
99%,271.615992,234.071602,290.345681,199.178034,310.513722,750.931340,516.494885,186.531212,168.827451,117.159780
max,286.190002,254.000000,323.228577,214.169998,327.691864,788.823792,541.057373,207.028473,175.074509,120.989998


In [93]:
df.to_csv("data/daily.csv", index=True)

In [94]:
prices = df.copy()
rets = prices.pct_change().add_prefix("ret_")

df = pd.concat([df, rets], axis=1).dropna()

TARGET_TICKER = "AAPL"
col = f"ret_{TARGET_TICKER}"
df["target"] = (df[col].shift(-1) > 0).astype(int)
df = df.drop(columns=[c for c in df.columns if "target" in c.lower()], errors="ignore")
df = df.dropna()

df.head()

,AAPL,AMZN,GOOGL,JNJ,JPM,META,MSFT,NVDA,PG,XOM,ret_AAPL,ret_AMZN,ret_GOOGL,ret_JNJ,ret_JPM,ret_META,ret_MSFT,ret_NVDA,ret_PG,ret_XOM
Date,,,,,,,,,,,,,,,,,,,,
2015-01-05,23.554735,15.1095,25.778227,76.418083,45.067196,76.654541,39.491920,0.474853,66.202065,56.332203,-0.028172,-0.020517,-0.019054,-0.006984,-0.031045,-0.016061,-0.009196,-0.016890,-0.004755,-0.027361
2015-01-06,23.556952,14.7645,25.142035,76.042580,43.898651,75.621750,38.912289,0.460457,65.900513,56.032726,0.000094,-0.022833,-0.024679,-0.004914,-0.025929,-0.013473,-0.014677,-0.030318,-0.004555,-0.005316
2015-01-07,23.887281,14.9210,25.068092,77.721291,43.965630,75.621750,39.406689,0.459257,66.246216,56.600468,0.014023,0.010600,-0.002941,0.022076,0.001526,0.000000,0.012705,-0.002605,0.005246,0.010132
2015-01-08,24.805077,15.0230,25.155436,78.332405,44.948116,77.637688,40.565952,0.476533,67.003761,57.542580,0.038422,0.006836,0.003484,0.007863,0.022347,0.026658,0.029418,0.037617,0.011435,0.016645
2015-01-09,24.831692,14.8465,24.848255,77.264793,44.166595,77.200737,40.224998,0.478453,66.378586,57.461452,0.001073,-0.011749,-0.012211,-0.013629,-0.017387,-0.005628,-0.008405,0.004028,-0.009330,-0.001410


In [102]:
df = df.sort_index()
assert isinstance(df.index, pd.DatetimeIndex)

y = df["ret_AAPL"].shift(-1)     # target = jutrzejszy zwrot AAPL
X = df.drop(columns=["ret_AAPL"])  # cechy = wszystko poza ret_AAPL

X = X.iloc[:-1]
y = y.iloc[:-1]

assert X.index.equals(y.index)

In [103]:
TRAIN_END = "2021-12-31"
VAL_END   = "2023-12-31"

X_train = X.loc[:TRAIN_END]
y_train = y.loc[:TRAIN_END]

X_val = X.loc["2022-01-01":VAL_END]
y_val = y.loc["2022-01-01":VAL_END]

X_test = X.loc["2024-01-01":]
y_test = y.loc["2024-01-01":]

assert X_val.index.equals(y_val.index)
assert X_test.index.equals(y_test.index)

print(len(X_train), len(X_val), len(X_test))

1762 501 500


In [105]:
model = HistGradientBoostingRegressor(
    max_depth=4,
    learning_rate=0.05,
    max_iter=300,
    random_state=42,
)

model.fit(X_train, y_train)

val_pred = model.predict(X_val)
test_pred = model.predict(X_test)

print("VAL MAE:", mean_absolute_error(y_val, val_pred))
print("VAL RMSE:", np.sqrt(mean_squared_error(y_val, val_pred)))
print("TEST MAE:", mean_absolute_error(y_test, test_pred))
print("TEST RMSE:", np.sqrt(mean_squared_error(y_test, test_pred)))

VAL MAE: 0.014559768627805156
VAL RMSE: 0.01954237599007647
TEST MAE: 0.012255383166204045
TEST RMSE: 0.018188925489168912


In [106]:
N = 30
X_all = X
pred_all = pd.Series(model.predict(X_all), index=X_all.index, name="pred_ret_AAPL_next")

forecast = pd.DataFrame({
    "feature_date": X_all.index[-N:],
    "forecast_date": [X_all.index[X_all.index.get_loc(d)+1] if X_all.index.get_loc(d) < len(X_all.index)-1 else pd.NaT
                      for d in X_all.index[-N:]],
    "pred_ret_AAPL_next": pred_all.iloc[-N:].values
})

forecast.tail(10)

,feature_date,forecast_date,pred_ret_AAPL_next
20,2025-12-15,2025-12-16,0.000554
21,2025-12-16,2025-12-17,0.006729
22,2025-12-17,2025-12-18,0.003585
23,2025-12-18,2025-12-19,0.003504
24,2025-12-19,2025-12-22,0.002974
25,2025-12-22,2025-12-23,-0.000770
26,2025-12-23,2025-12-24,0.001151
27,2025-12-24,2025-12-26,0.003559
28,2025-12-26,2025-12-29,0.007108
29,2025-12-29,NaT,0.005117
